# `mkdocs` plugin for jupyter notebooks

i think i want more control of how `mkdocs` renders notebooks.
i've been using [`mkdocs-jupyter`][mkdocs-jupyter] for a while and it is great,
but i need more knobs.

my particular need is to configure `nbconvert` exporters with more fine grain control than what [`mkdocs-jupyter`][mkdocs-jupyter] offers.
the major difference is we are going to target markdown output rather than html.

[mkdocs-jupyter]: https://github.com/danielfrg/mkdocs-jupyter
[plugin]: https://www.mkdocs.org/dev-guide/plugins/

## checklist for successfully integrating the plugin

steps to adding a [`mkdocs` plugin][plugin] to this the `tonyfast` project:

- [x] add plugin to `mkdocs.yml`

  ```yaml
  plugins:
      - markdown_notebook
  ```
  
- [x] define plugin entry point for `tonyfast`

   ```toml
   [project.entry-points."mkdocs.plugins"]
   markdown_notebook = "tonyfast.mkdocs:MarkdownNotebook"
   ```

- [x] build the `MarkdownNotebook` plugin 
- [x] integrate the plugin

   ```python
   from tonyfast.mkdocs import MarkdownNotebook
   ```
   
- [ ] add and improve the `nbconvert` export display renderers

[mkdocs-jupyter]: https://github.com/danielfrg/mkdocs-jupyter
[plugin]: https://www.mkdocs.org/dev-guide/plugins/

## building the [`mkdocs` plugin][plugin]

[mkdocs-jupyter]: https://github.com/danielfrg/mkdocs-jupyter
[plugin]: https://www.mkdocs.org/dev-guide/plugins/

In [1]:
    import json, nbconvert, nbformat, pathlib, mkdocs.plugins, warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

the [`mkdocs` plugin]

In [2]:
    class MarkdownNotebook(mkdocs.plugins.BasePlugin):
        exporter = nbconvert.exporters.HTMLExporter(template_file="html-md")
        from jinja2 import DictLoader
        for loader in exporter.environment.loader.loaders:
            if isinstance(loader, DictLoader):
                break
        del DictLoader
        
        config_scheme = (
            # ('foo', mkdocs.config.config_options.Type(str, default='a default value')),
        )
            
        def on_page_read_source(self, page, config):
            if page.file.src_uri.endswith((".ipynb", )):
                body = pathlib.Path(page.file.abs_src_path).read_text()
                nb = nbformat.v4.reads(body)
                prepare_notebook(nb)
                return "\n".join((
                    "---", json.dumps(nb.metadata), "---", # add metadata as front matter
                    self.exporter.from_notebook_node(nb)[0]            
                ))
        def on_page_markdown(self, markdown, page, config, files):
            import markdown
            title = markdown.Markdown(extensions=config['markdown_extensions']).convert(page.title)
            page.title = title[len("<p>"):-len("</p>")].strip().replace("code>", "pre>")

we trick `mkdocs` into thinking notebook files are markdown extensions.

In [3]:
    mkdocs.utils.markdown_extensions += ".ipynb", # this feels naughty.

it seems the `nbconvert` markdown templates haven't gotten some love in a while. in prepare notebook, we update the notebook form so that it works with the current `jinja2` templates we need.

In [4]:
    def prepare_notebook(nb):
        for cell in nb.cells:
            cell.source = "".join(cell.source)
            for output in cell.get("outputs", ""):
                if "text" in output:
                    output.text = "".join(output.text)
                if "data" in output:
                    for k, v in output["data"].items():
                        if k.startswith(("text",)):
                            output.data[k] = "".join(v)

## modifications to the classic html template

these changes let `mkdocs` use its machinery on as much markdown as possible.

In [5]:
    MarkdownNotebook.loader.mapping["html-md"] = """{%- extends 'classic/base.html.j2' -%}
    {% block in_prompt %}
    {% endblock in_prompt %}

    {% block output_prompt %}
    {%- endblock output_prompt %}

    {% block input %}
    ```
    {%- if 'magics_language' in cell.metadata  -%}
        {{ cell.metadata.magics_language}}
    {%- elif 'name' in nb.metadata.get('language_info', {}) -%}
        {{ nb.metadata.language_info.name }}
    {%- endif %}
    {{ cell.source}}
    ```
    {% endblock input %}

    {% block markdowncell scoped %}
    {{ cell.source }}
    {% endblock markdowncell %}

    {% block data_markdown scoped %}
    {{ output.data['text/markdown'] }}
    {% endblock data_markdown %}"""